In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = 23137010 # 店舗ID
SPREADSHEET_KEY = '1zXfnKTlawKwa8ej-6NEvibPaNd4exq6SKOOWWHzrmw0' # スプレッドシートのID
pstart = 521 # パチンコの1台目
sstart = 1 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value
s2row = worksheet.cell(11,  30).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []
s2table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "125":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "125":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "537":
                    s5table.append(li)
                if unit == "250":
                    s2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "537":
                        s5table.append(li)
                    if unit == "250":
                        s2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1.25RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})
wb.values_append("'S2.5RAW'!F" + str(int(s2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s2table})

['08/19', '025665', '521', '41795.0', '5719.0', '30', '8']
['08/19', '025665', '522', '41282.0', '31899.0', '48', '12']
['08/19', '025665', '523', '38205.0', '-902.0', '25', '8']
['08/19', '025665', '524', '40000.0', '1506.0', '26', '6']
['08/19', '025665', '525', '32051.0', '-16249.0', '10', '6']
['08/19', '025665', '526', '47179.0', '-12337.0', '22', '8']
['08/19', '025665', '527', '40769.0', '904.0', '25', '6']
['08/19', '025665', '528', '39231.0', '-19258.0', '11', '3']
['08/19', '025665', '529', '35385.0', '18357.0', '32', '5']
['08/19', '025665', '530', '47949.0', '-6018.0', '24', '4']
['08/19', '025665', '531', '46923.0', '-4814.0', '26', '9']
['08/19', '025665', '532', '39487.0', '17455.0', '34', '5']
['08/19', '025665', '533', '41795.0', '-18656.0', '15', '8']
['08/19', '025665', '534', '38718.0', '-29791.0', '5', '5']
['08/19', '025665', '535', '38462.0', '-21064.0', '10', '4']
['08/19', '025665', '536', '39744.0', '-16550.0', '15', '8']
['08/19', '025665', '537', '44359.0', 

['08/19', '025489', '658', '20513.0', '603.0', '9', '3']
['08/19', '025489', '659', '55641.0', '-2707.0', '21', '8']
['08/19', '025489', '660', '40769.0', '-6619.0', '13', '6']
['08/19', '025489', '661', '37692.0', '12038.0', '23', '7']
['08/19', '025489', '662', '32564.0', '1205.0', '14', '8']
['08/19', '025489', '663', '29487.0', '603.0', '13', '5']
['08/19', '025489', '664', '25128.0', '6621.0', '15', '4']
['08/19', '025489', '665', '37949.0', '18056.0', '27', '5']
['08/19', '025489', '666', '56410.0', '-21064.0', '11', '7']
['08/19', '025489', '667', '30769.0', '-5717.0', '10', '4']
['08/19', '025489', '668', '34103.0', '14746.0', '23', '8']
['08/19', '025489', '669', '44359.0', '-2707.0', '16', '7']
['08/19', '025489', '670', '36410.0', '-4212.0', '13', '5']
['08/19', '025489', '671', '44615.0', '-902.0', '18', '7']
['08/19', '025489', '672', '36410.0', '2709.0', '17', '5']
['08/19', '025489', '673', '12051.0', '-5717.0', '2', '2']
['08/19', '025489', '674', '42564.0', '3612.0', '

['08/19', '025818', '795', '35897.0', '-10832.0', '15', '6']
['08/19', '025879', '796', '38205.0', '25580.0', '60', '10']
['08/19', '025879', '797', '42051.0', '-17152.0', '20', '7']
['08/19', '025879', '798', '27436.0', '4515.0', '30', '8']
['08/19', '025879', '799', '44872.0', '2709.0', '46', '14']
['08/19', '025879', '800', '39487.0', '-17754.0', '17', '6']
['08/19', '025807', '801', '39487.0', '-5717.0', '34', '9']
['08/19', '025807', '802', '45385.0', '-3610.0', '40', '13']
['08/19', '025544', '803', '45385.0', '12038.0', '99', '18']
['08/19', '025544', '804', '50000.0', '-12036.0', '96', '19']
['08/19', '025844', '805', '41282.0', '16853.0', '59', '16']
['08/19', '025844', '806', '43590.0', '-5416.0', '36', '11']
['08/19', '025707', '807', '43590.0', '-29791.0', '11', '8']
['08/19', '025871', '808', '31282.0', '-3610.0', '20', '10']
['08/19', '025667', '809', '48718.0', '17154.0', '44', '15']
['08/19', '025797', '810', '34359.0', '-15346.0', '48', '12']
['08/19', '025473', '811',

['08/19', '025576', '971', '36154.0', '8427.0', '55', '10']
['08/19', '025637', '972', '46410.0', '8728.0', '61', '30']
['08/19', '025785', '973', '36154.0', '15348.0', '56', '56']
['08/19', '025835', '974', '32051.0', '-3911.0', '93', '17']
['08/19', '025689', '975', '35897.0', '11737.0', '60', '15']
['08/19', '025831', '976', '36923.0', '-5115.0', '39', '13']
['08/19', '025576', '977', '35385.0', '-10231.0', '28', '14']
['08/19', '025664', '978', '28974.0', '-3008.0', '50', '12']
['08/19', '025891', '979', '58205.0', '-5416.0', '87', '43']
['08/19', '025685', '980', '31795.0', '603.0', '99', '14']
['08/19', '025685', '981', '39231.0', '-5115.0', '99', '19']
['08/19', '025659', '982', '41282.0', '2408.0', '50', '20']
['08/19', '025835', '983', '34872.0', '-6319.0', '99', '17']
['08/19', '025780', '984', '30256.0', '-6018.0', '47', '21']
['08/19', '025572', '985', '43333.0', '-13842.0', '28', '15']
['08/19', '025523', '986', '35897.0', '9029.0', '60', '19']
['08/19', '025774', '987', '

['08/19', '025835', '1107', '24359.0', '-1203.0', '87', '15']
['08/19', '025866', '1108', '5641.0', '4515.0', '23', '2']
['08/19', '025722', '1109', '50769.0', '-15346.0', '41', '18']
['08/19', '025841', '1110', '17692.0', '603.0', '16', '5']
['08/19', '025875', '1111', '15385.0', '-9629.0', '8', '8']
['08/19', '025875', '1112', '19231.0', '1205.0', '22', '22']
['08/19', '025868', '1113', '8974.0', '603.0', '12', '6']
['08/19', '025885', '1114', '14872.0', '-3309.0', '38', '5']
['08/19', '025855', '1115', '19744.0', '9931.0', '50', '12']
['08/19', '025875', '1116', '8205.0', '6621.0', '15', '15']
['08/19', '025590', '1117', '26667.0', '5719.0', '34', '34']
['08/19', '025590', '1118', '1282.0', '-1203.0', '1', '1']
['08/19', '025590', '1119', '9487.0', '-1504.0', '8', '8']
['08/19', '025590', '1120', '22564.0', '-3911.0', '21', '21']
['08/19', '025897', '1121', '38974.0', '-4212.0', '26', '6']
['08/19', '025897', '1122', '44615.0', '10533.0', '41', '12']
['08/19', '025871', '1123', '123

...

['08/19', '120122', '98', '6166', '-2454.0', '47', '13', '--']
['08/19', '120122', '99', '4763', '-2778.0', '29', '8', '--']
['08/19', '120122', '100', '3747', '-3148.0', '16', '5', '--']
['08/19', '120122', '101', '4604', '-1157.0', '41', '8', '--']
['08/19', '120122', '102', '3890', '-1528.0', '36', '7', '--']
['08/19', '120122', '103', '5544', '1389.0', '72', '15', '--']
['08/19', '120122', '104', '5533', '1713.0', '69', '13', '--']
['08/19', '120122', '105', '5575', '-1481.0', '44', '18', '--']
['08/19', '120122', '106', '7112', '-3287.0', '51', '14', '--']
['08/19', '120122', '107', '4045', '1296.0', '57', '10', '--']
['08/19', '120122', '108', '6567', '1157.0', '83', '22', '--']
['08/19', '120122', '109', '3682', '3287.0', '65', '9', '--']
['08/19', '120122', '110', '5113', '1111.0', '66', '13', '--']
['08/19', '120122', '111', '5807', '-1806.0', '47', '11', '--']
['08/19', '120122', '112', '6179', '-2083.0', '50', '16', '--']
['08/19', '120125', '113', '6956', '-556.0', '22', '1

['08/19', '120110', '230', '5026', '-556.0', '29', '13', '--']
['08/19', '120110', '231', '6049', '-2963.0', '22', '18', '--']
['08/19', '120110', '232', '5140', '648.0', '36', '12', '--']
['08/19', '120110', '233', '3334', '9676.0', '65', '14', '--']
['08/19', '120110', '234', '6473', '-2407.0', '31', '14', '--']
['08/19', '120110', '235', '5507', '-324.0', '31', '17', '--']
['08/19', '120110', '236', '5147', '1944.0', '44', '11', '--']
['08/19', '120110', '237', '4752', '-1204.0', '27', '8', '--']
['08/19', '120110', '238', '4953', '1806.0', '36', '20', '--']
['08/19', '120110', '239', '3726', '1852.0', '34', '6', '--']
['08/19', '120110', '240', '5957', '-2778.0', '24', '17', '--']
['08/19', '120110', '241', '5633', '-1250.0', '30', '12', '--']
['08/19', '120110', '242', '4352', '787.0', '27', '18', '--']
['08/19', '120110', '243', '5473', '278.0', '35', '17', '--']
['08/19', '120110', '244', '5099', '1296.0', '41', '10', '--']
['08/19', '120110', '245', '6510', '-4907.0', '17', '11

['08/19', '120110', '363', '4711', '1991.0', '37', '18', '--']
['08/19', '120110', '364', '4562', '4120.0', '48', '12', '--']
['08/19', '120110', '365', '6343', '-4167.0', '22', '14', '--']
['08/19', '120110', '366', '5996', '231.0', '38', '19', '--']
['08/19', '120073', '404', '8307', '1111.0', '0', '26', '59']
['08/19', '120073', '405', '4334', '-3009.0', '0', '15', '16']
['08/19', '120073', '406', '6105', '-417.0', '0', '14', '37']
['08/19', '120073', '407', '6826', '-3148.0', '0', '23', '33']
['08/19', '120073', '408', '7978', '185.0', '0', '23', '58']
['08/19', '120073', '409', '7456', '-324.0', '0', '27', '49']
['08/19', '120073', '410', '7210', '833.0', '0', '24', '52']
['08/19', '120073', '411', '8787', '4954.0', '0', '17', '82']
['08/19', '120073', '412', '8052', '6204.0', '0', '23', '85']
['08/19', '120073', '413', '5732', '-648.0', '0', '20', '35']
['08/19', '120073', '414', '5777', '-1667.0', '0', '14', '32']
['08/19', '120073', '415', '7911', '3519.0', '0', '17', '67']
['0

['08/19', '120117', '932', '6111', '-1991.0', '0', '19', '83']
['08/19', '120127', '933', '6857', '-231.0', '0', '0', '12']
['08/19', '120126', '934', '8551', '-231.0', '0', '20', '47']
['08/19', '120034', '935', '5752', '139.0', '0', '12', '42']
['08/19', '120108', '936', '9999', '-2870.0', '0', '0', '49']
['08/19', '120077', '937', '6725', '2639.0', '0', '28', '60']
['08/19', '120077', '938', '7056', '2500.0', '0', '37', '67']
['08/19', '120077', '939', '7337', '-2361.0', '0', '19', '50']
['08/19', '120077', '940', '7807', '1481.0', '0', '45', '82']
['08/19', '120096', '941', '7936', '-2176.0', '24', '17', '--']
['08/19', '120096', '942', '8647', '1713.0', '41', '25', '--']
['08/19', '120126', '943', '7214', '417.0', '0', '16', '41']
['08/19', '120127', '944', '8034', '-2870.0', '0', '2', '17']
['08/19', '120094', '945', '5969', '-2685.0', '0', '6', '77']
['08/19', '120120', '946', '5310', '-3796.0', '0', '0', '22']
['08/19', '120034', '947', '6604', '-1343.0', '0', '9', '49']
['08/1

{'spreadsheetId': '1zXfnKTlawKwa8ej-6NEvibPaNd4exq6SKOOWWHzrmw0',
 'updates': {'spreadsheetId': '1zXfnKTlawKwa8ej-6NEvibPaNd4exq6SKOOWWHzrmw0',
  'updatedRange': 'S20RAW!F81892:M82360',
  'updatedRows': 469,
  'updatedColumns': 8,
  'updatedCells': 3752}}